In [1]:
import numpy as np

from fpcross import ij, Grid, Solver, Model, Check

ij()

Start | 11:36AM MSK on Oct 22, 2019 | python 3.7.1    |
-------------------------------------------------------


Let solve
$$
    \frac{\partial \psi}{\partial t}
    =
    - A \psi,
    \quad
    \psi(0) = \psi_0,
    \quad
    x \in \Omega,
$$
where
$$
    A \psi
    =
    - \epsilon \Delta \psi
    + div({\psi v}),
$$
with $\epsilon \in R$ and $v \in R^{d}$.

Note that for the case of potential field $v = grad(\phi)$ the analytical solution of the stationary problem is given as $\psi = C e^{-\frac{\phi}{\epsilon}}$, where the constant $C$ is defined to satisfy the normalization condition $\int_{\Omega} \psi \, d x = 1$.

Consider three-dimensional equation (dumbbell model)
$$
    \epsilon=\frac{1}{2},
    \quad
    v = K x - \frac{1}{2} grad(\phi),
    \quad
    x \in [-a, a]^3,
$$
where
$$
    \phi = \frac{1}{2} x^2 + \frac{\alpha}{p^3} e^{-\frac{x^2}{2 p^2}},
$$
and
$$
    K = \beta \begin{bmatrix}
        0 & 1 & 0 \\
        0 & 0 & 0 \\
        0 & 0 & 0
    \end{bmatrix}.
$$

We have
$$
    v
    =
    [v_1, v_2, v_3]^T =
    K x
    - \frac{1}{2} x
    + \frac{\alpha}{2 p^5} e^{-\frac{x^2}{2 p^2}} x,
$$

$$
    \frac{\partial v_1}{\partial x_1}
    =
    - \frac{1}{2}
    + \frac{\alpha}{2 p^5} e^{-\frac{x^2}{2 p^2}}
    - \frac{\alpha}{2 p^7} e^{-\frac{x^2}{2 p^2}} x_1^2,
$$

$$
    \frac{\partial v_2}{\partial x_2}
    =
    - \frac{1}{2}
    + \frac{\alpha}{2 p^5} e^{-\frac{x^2}{2 p^2}}
    - \frac{\alpha}{2 p^7} e^{-\frac{x^2}{2 p^2}} x_2^2,
$$

$$
    \frac{\partial v_3}{\partial x_3}
    =
    - \frac{1}{2}
    + \frac{\alpha}{2 p^5} e^{-\frac{x^2}{2 p^2}}
    - \frac{\alpha}{2 p^7} e^{-\frac{x^2}{2 p^2}} x_3^2.
$$

We select parameter values as in the paper
$$
    \beta = 1,
    \quad
    \alpha = 0.1,
    \quad
    p = 0.5,
    \quad
    x \in [-10, 10]^3,
    \quad
    t \in [0, 10],
    \quad
    \epsilon = 10^{-6}.
$$

In [3]:
a = 0.1
b = 1.
p = 0.5
s = 1.
D = 0.5 * 1.E-6

class Model_(Model):

    def d(self):
        return 3
    
    def D(self):
        return D

    def f0(self, X, t):
        mult = a / 2. / p**5
        mult*= np.exp(-np.sum(X*X, axis=0) / 2. / p**2)
        
        return np.vstack([
            -0.5 * X[0, :] + mult * X[0, :] + b * X[1, :],
            -0.5 * X[1, :] + mult * X[1, :],
            -0.5 * X[2, :] + mult * X[2, :]
        ])

    def f1(self, X, t):
        mult = a / 2. / p**5
        mult*= np.exp(-np.sum(X*X, axis=0) / 2. / p**2)
        
        return np.vstack([
            -0.5 + mult - mult / p**2 * X[0, :],
            -0.5 + mult - mult / p**2 * X[1, :],
            -0.5 + mult - mult / p**2 * X[2, :]
        ])

    def r0(self, X):
        a = 2. * s
        r = np.exp(-np.sum(X*X, axis=0) / a) / (np.pi * a)**1.5
        return r.reshape(-1)

In [4]:
SL = Solver(
    TG=Grid(d=1, n=10, l=[+ 0., +10.], kind='u'),
    SG=Grid(d=3, n=21, l=[-10., +10.], kind='c'),
    MD=Model_(),
).init().prep().calc().info()

Solve: 100%|█| 9/9 [01:38<00:00, 11.05s/step, | At T=1.0e+01 : n=-9.70e+03 rhs=8.46e-01]

------------------ Solver
Format    : 3D, NP [order=2]
Time prep : 1.67e+00 
Time calc : 9.85e+01 
...  init : 4.66e-03 
...  diff : 8.76e-01 
...  conv : 7.16e-01 
...  post : 9.68e+01 
...  last : 3.06e-03 


In [6]:
SL = Solver(
    TG=Grid(d=1, n=100, l=[+ 0., +10.], kind='u'),
    SG=Grid(d=3, n=21, l=[-10., +10.], kind='c'),
    MD=Model_(),
    eps=1.E-3, with_tt=True
).init().prep().calc().info()

Solve: 100%|█| 99/99 [00:31<00:00,  3.17step/s, | At T=1.0e+01 : r=5.71e+00 n=-1.16e+04 rhs=1.18e+01]

------------------ Solver
Format    : 3D, TT, eps= 1.00e-03 [order=2]
Time prep : 2.57e-03 
Time calc : 3.10e+01 
...  init : 2.15e-02 
...  diff : 1.83e-01 
...  conv : 2.96e+01 
...  post : 1.10e+00 
...  last : 8.75e-04 
